Feature engineering. 
 Feature engineering is a bit of an art and having knowledge in the specific domain (in this case real estate) can help you create better features. In this mission, we'll focus on some domain-independent strategies that work for all problems.

In the first half of this mission, we'll focus only on columns that contain no missing values but still aren't in the proper format to use in a linear regression model. In the latter half of this mission, we'll explore some ways to deal with missing values.

Amongst the columns that don't contain missing

In [11]:
import pandas as pd

data = pd.read_csv('AmesHousing.txt', delimiter="\t")
train = data[0:1460]
test = data[1460:]

train_null_counts = train.isnull().sum()
#print(train_null_counts)
df_no_mv=train[train_null_counts[train_null_counts == 0].index]

You'll notice that some of the columns in the data frame df_no_mv contain string values. If these columns contain only a limited set of uniuqe values, they're known as categorical features. As the name suggests, a categorical feature groups a specific training example into a specific category. Here are some examples from the dataset:

In [12]:

text_cols=df_no_mv.select_dtypes(include=['object']).columns

for col in text_cols:
    print(col+":", len(train[col].unique()))
    
for col in text_cols:
    train[col]=train[col].astype('category')
train['Utilities'].cat.codes.value_counts()    

('MS Zoning:', 6)
('Street:', 2)
('Lot Shape:', 4)
('Land Contour:', 4)
('Utilities:', 3)
('Lot Config:', 5)
('Land Slope:', 3)
('Neighborhood:', 26)
('Condition 1:', 9)
('Condition 2:', 6)
('Bldg Type:', 5)
('House Style:', 8)
('Roof Style:', 6)
('Roof Matl:', 5)
('Exterior 1st:', 14)
('Exterior 2nd:', 16)
('Exter Qual:', 4)
('Exter Cond:', 5)
('Foundation:', 6)
('Heating:', 6)
('Heating QC:', 4)
('Central Air:', 2)
('Electrical:', 4)
('Kitchen Qual:', 5)
('Functional:', 7)
('Paved Drive:', 3)
('Sale Type:', 9)
('Sale Condition:', 5)


C:\Users\kknagara\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    1457
2       2
1       1
dtype: int64

In [13]:
dummy_cols = pd.get_dummies(train[text_cols]) 
train = train.drop(text_cols, axis=1) 
train = pd.concat([train, dummy_cols], axis=1)

In [14]:
train.head(5)

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Alley,Overall Qual,Overall Cond,Year Built,Year Remod/Add,...,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_WD,Sale Condition_Abnorml,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,1,526301100,20,141.0,31770,NaN,6,5,1960,1960,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,2,526350040,20,80.0,11622,NaN,5,6,1961,1961,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,526351010,20,81.0,14267,NaN,6,6,1958,1958,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,4,526353030,20,93.0,11160,NaN,7,5,1968,1968,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,5,527105010,60,74.0,13830,NaN,5,5,1997,1998,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Transforming Improper Numerical Features
4: Transforming Improper Numerical Features

In the last few screens, we focused on categorical values that were represented as text columns. Some of the numerical columns in the data set are also categorical and only have a limited set of unique values. We won't explicitly explore those coumns in this mission, but the feature transformation process is the same if the numbers used in those categories have no numerical meaning.

Let's now look at numerical features that aren't categorical, but whose numerical representation needs to be improved. We'll focus on the Year Remod/Add and Year Built columns:

>>> train[['Year Remod/Add', 'Year Built']]

0   1960    1960

1   1961    1961

2   1958    1958

3   1968    1968

4   1998    1997

...

The two main issues with these features are:

    Year values aren't representative of how old a house is
    The Year Remod/Add column doesn't actually provide useful information for a linear regression model

The challenge with year values like 1960 and 1961 is that they don't do a good capture how old a house is. For example, a house that was built in 1960 but sold in 1980 was sold in half the time one built in 1960 and sold in 2000. Instead of the years certain events happened, we want the difference between those years. We should create a new column that's the difference between both of these columns.

For this particular piece of information (years until remodeled), this is a sensible approach. Domain knowledge can help you understand how to best transform features to represent information well for a linear model. If you're ever confused about a feature or how it should be represented, reading scientific papers or posts by researchers in the specific domain is critical.

In [15]:
train['years_until_remod']=train['Year Remod/Add'] - train['Year Built']

5: Missing Values

In the next few screens, we'll focus on handling columns with missing values. When values are missing in a column, there are two main approaches we can take:

    Remove rows containing missing values for specific columns
        Pro: Rows containing missing values are removed, leaving only clean data for modeling
        Con: Entire observations from the training set are removed, which can reduce overall prediction accuracy
    Impute (or replace) missing values using a descriptive statistic from the column
        Pro: Missing values are replaced with potentially similar estimates, preserving the rest of the observation in the model.
        Con: Depending on the approach, we may be adding noisy data for the model to learn

Given that we only have 1460 training examples (with ~80 potentially useful features), we don't want to remove any of these rows from the dataset. Let's instead focus on imputation techniques.

We'll focus on columns that contain at least 1 missing value but less than 365 missing values (or 25% of the number of rows in the training set). There's no strict threshold, and many people instead use a 50% cutoff (if half the values in a column are missing, it's automatically dropped). Having some domain knowledge can help with determining an acceptable cutoff value.

In [16]:
import pandas as pd

data = pd.read_csv('AmesHousing.txt', delimiter="\t")
train = data[0:1460]
test = data[1460:]

train_null_counts = train.isnull().sum()
df_missing_values = train[train_null_counts[(train_null_counts>0) & (train_null_counts<584)].index]

print(df_missing_values.isnull().sum())
print(df_missing_values.dtypes)

Lot Frontage      249
Mas Vnr Type       11
Mas Vnr Area       11
Bsmt Qual          40
Bsmt Cond          40
Bsmt Exposure      41
BsmtFin Type 1     40
BsmtFin SF 1        1
BsmtFin Type 2     41
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
Bsmt Full Bath      1
Bsmt Half Bath      1
Garage Type        74
Garage Yr Blt      75
Garage Finish      75
Garage Qual        75
Garage Cond        75
dtype: int64
Lot Frontage      float64
Mas Vnr Type       object
Mas Vnr Area      float64
Bsmt Qual          object
Bsmt Cond          object
Bsmt Exposure      object
BsmtFin Type 1     object
BsmtFin SF 1      float64
BsmtFin Type 2     object
BsmtFin SF 2      float64
Bsmt Unf SF       float64
Total Bsmt SF     float64
Bsmt Full Bath    float64
Bsmt Half Bath    float64
Garage Type        object
Garage Yr Blt     float64
Garage Finish      object
Garage Qual        object
Garage Cond        object
dtype: object


In [18]:
float_cols = df_missing_values.select_dtypes(include=['float'])
float_cols = float_cols.fillna(df_missing_values.mean())
print(float_cols.isnull().sum())

Lot Frontage      0
Mas Vnr Area      0
BsmtFin SF 1      0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
Garage Yr Blt     0
dtype: int64
